Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [143]:
from random import random
from math import ceil
from functools import reduce
from collections import namedtuple, deque
from queue import PriorityQueue

import numpy as np
import copy
from tqdm.auto import tqdm

In [144]:
State = namedtuple('State', ['taken', 'not_taken'])


def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )


def goal_check(state):
    return np.all(covered(state))

In [145]:
PROBLEM_SIZE = 30
NUM_SETS = 80
SETS = tuple(np.array([random() < 0.1 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"

## A*

In [146]:
def h(state):
    largest_set_size = max(sum(s) for s in SETS) # I take the set with the biggest number of true
    missing_size = PROBLEM_SIZE - sum(covered(state)) # number of missing tiles
    optimistic_estimate = ceil(missing_size / largest_set_size) # minimum number of missing tiles
    return optimistic_estimate


def h2(state):
    already_covered = covered(state) #already covered elements
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS) # I take the tile with the max number of missing elements
    missing_size = PROBLEM_SIZE - sum(already_covered) # elements i still have to cover
    optimistic_estimate = ceil(missing_size / largest_set_size) # minimum number of missing tiles
    return optimistic_estimate


def h3(state):
    already_covered = covered(state) #already covered elements
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered) # number of elements not covered
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True) # i sort the remaining tiles from the biggest remaining n of elements occupied to the last
    taken = 1
    while sum(candidates[:taken]) < missing_size: # scorro il vettore dei tile per stimare al meglio quante ne mancano
        taken += 1
    return taken

# worked with Alexandre and Paolo after this comment

def h4(state):

    updated_state = copy.deepcopy(state)
    already_covered = covered(state) #already covered elements
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered) # number of elements not covered
    candidates = sorted(updated_state.not_taken , key = lambda x: sum(np.logical_and(x, np.logical_not(already_covered))), reverse=True) # i sort the remaining tiles from the biggest remaining n of elements occupied to the last
    taken = 1
    candidates



    #start of while
    while sum(candidates[:taken]) < missing_size: # scorro il vettore dei tile per stimare al meglio quante ne mancano

        updated_state = State(updated_state.taken ^ {candidates[0]}, updated_state.not_taken ^ {candidates[0]},)
        already_covered = covered(updated_state) #already covered elements

        missing_size = PROBLEM_SIZE - sum(already_covered) # number of elements not covered
        candidates = sorted(updated_state.not_taken , key = lambda x: sum(np.logical_and(x, np.logical_not(already_covered))), reverse=True) # i sort the remaining tiles from the biggest remaining n of elements occupied to the last

        taken += 1
    return taken

def h5(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    heuri = 1
    taken = 1
    a = sum(candidates[:taken])
    while sum(candidates[:taken]) < missing_size:
        if sum(candidates[:taken]) > a:
            a = sum(candidates[:taken])
            heuri += 1
        if sum(candidates[:taken]) == a:
            heuri +=0.9
        taken += 1
    return heuri

def h6(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted(state.not_taken , key = lambda x: sum(np.logical_and(x, np.logical_not(already_covered))), reverse=True) # i sort the remaining tiles from the biggest remaining n of elements occupied to the last
    heuri = 1
    taken = 1
    a = sum(candidates[:taken])
    while sum(candidates[:taken]) < missing_size:
        if sum(candidates[:taken]) > a:
            a = sum(candidates[:taken])
            heuri += 1
        if sum(candidates[:taken]) == a:
            heuri +=0.9
        taken += 1
    return heuri

def h7(state):
    updated_state = copy.deepcopy(state)
    already_covered = covered(state) #already covered elements
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered) # number of elements not covered
    candidates = sorted(updated_state.not_taken , key = lambda x: sum(np.logical_and(SETS[x], np.logical_not(already_covered))), reverse=True) # i sort the remaining tiles from the biggest remaining n of elements occupied to the last
    #print(sum(np.logical_and(candidates[0], np.logical_not(already_covered))) , sum(np.logical_and(candidates[1], np.logical_not(already_covered))) )
    updated_state = State(updated_state.taken ^ {candidates[0]}, updated_state.not_taken ^ {candidates[0]},)

    already_covered = covered(updated_state) #already covered elements
    if np.all(already_covered):
        return 1
    missing_size = PROBLEM_SIZE - sum(already_covered) # number of elements not covered
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in (SETS[i] for i in updated_state.not_taken)), reverse=True) # i sort the remaining tiles from the biggest remaining n of elements occupied to the last
    taken = 2

    #start of while
    while sum(candidates[:taken-1]) < missing_size: # scorro il vettore dei tile per stimare al meglio quante ne mancano

        taken += 1
    return taken

def h8(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    heuri = 1
    taken = 1
    best_candidate_sum = sum(candidates[:taken])
    while sum(candidates[:taken]) < missing_size:
        if sum(candidates[:taken]) > best_candidate_sum:
            best_candidate_sum = sum(candidates[:taken])
            heuri += 1
        if sum(candidates[:taken]) == best_candidate_sum:
            heuri +=1
        if sum(candidates[:taken]) < best_candidate_sum:
            heuri +=0.8
        taken += 1
    return heuri

#upgrade of h7 wich is very fast by optimizing the selection of the candidate

def h9(state):
    updated_state = copy.deepcopy(state)
    already_covered = covered(updated_state)  # already covered elements
    if np.all(already_covered):
        return 0

    missing_size = PROBLEM_SIZE - np.sum(already_covered)  # number of elements not covered

    candidates = sorted(updated_state.not_taken, key=lambda x: np.sum(SETS[x] & ~already_covered), reverse=True)

    taken = 0
    # search the best candidate by comparing the number of missing elements covered by the candidate
    while missing_size > 0:
        if taken >= len(candidates):
            return float('inf')  # No more candidates, cannot cover the missing elements

        selected_candidate = candidates[taken]
        updated_state.taken.add(selected_candidate)
        updated_state.not_taken.remove(selected_candidate)
        already_covered |= SETS[selected_candidate]
        missing_size = PROBLEM_SIZE - np.sum(already_covered)
        taken += 1

    return taken


def f(state):
    return len(state.taken) + h9(state)

In [147]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f(new_state), new_state))
            #print(f(new_state), new_state)

        _, current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

0it [00:00, ?it/s]

25it [00:00, 25.48it/s]

Solved in 25 steps (8 tiles)
